## Introducing Distributed Word Vectors



In [4]:
!pip install gensim

In [15]:
from google.colab import files
uploaded = files.upload()

Saving unlabeledTrainData.tsv.zip to unlabeledTrainData.tsv.zip


In [16]:
# !unzip labeledTrainData.tsv.zip
# !unzip testData.tsv.zip
!unzip unlabeledTrainData.tsv.zip

Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


In [13]:
!ls -la

total 91380
drwxr-xr-x 1 root root     4096 Aug 14 07:49 .
drwxr-xr-x 1 root root     4096 Aug 14 06:36 ..
drwx------ 4 root root     4096 Aug 14 06:38 .cache
drwxr-xr-x 3 root root     4096 Aug 14 06:38 .config
drwxr-xr-x 3 root root     4096 Aug 10 16:59 datalab
drwxr-xr-x 4 root root     4096 Aug 14 06:37 .forever
drwxr-xr-x 5 root root     4096 Aug 14 06:38 .ipython
-rw-r--r-- 1 root root 33556378 May  5 20:07 labeledTrainData.tsv
-rw-r--r-- 1 root root 13788274 Aug 14 07:35 labeledTrainData.tsv.zip
drwx------ 3 root root     4096 Aug 14 06:37 .local
-rw-r--r-- 1 root root 32724746 May  5 20:07 testData.tsv
-rw-r--r-- 1 root root 13457819 Aug 14 07:44 testData.tsv.zip


In [19]:
import pandas as pd

# Read data from files
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

# Verify the number of reviews that were read(100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews," \
     "and %d unlabeled reviews \n" %(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews,and 50000 unlabeled reviews 



In [0]:
# Import various modules fir string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
  # Function to convert a document to a sequence of words,
  # optionally removing stop words. Returns a list of words.
  # 
  # 1. Remove HTML
  review_text = BeautifulSoup(review).get_text()
  # 
  # 2. Remove non-letters
  review_text = re.sub("[^a-zA-Z]", " ", review_text)
  # 
  # 3. Convert words to lower case and split them
  words = review_text.lower().split()
  # 
  # 4. Optionally remove stop words (false by default)
  if remove_stopwords:
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
  # 
  # 5. Return a list of words
  return(words)

In [31]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [36]:
print(len(sentences))

795538


In [37]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [38]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


### Training and Saving My Model  
2core の Mac よりも5倍早く実行できた

In [41]:
# Import the build-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                   level=logging.INFO)

# Set values for various parameters
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

# Initialize and train the model(this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
                         size=num_features, min_count=min_word_count, \
                         window = context, sample = downsampling)

# If you don't plan to train the model any feature, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and
# sabe the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-08-14 10:33:06,354 : INFO : collecting all words and their counts
2018-08-14 10:33:06,356 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-14 10:33:06,442 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-08-14 10:33:06,516 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2018-08-14 10:33:06,595 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-08-14 10:33:06,680 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-08-14 10:33:06,757 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-08-14 10:33:06,828 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-08-14 10:33:06,911 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-08-14 10:33:07,000 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-08-14 10:33:07,085 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-08-14 10:33:07,157 : INFO : PROGRESS: at sentence #100000, processed 2226967 words, keeping 50207 word types
2018-08-14 10:33:07,231 : INFO : PROGRESS: at sentence #110000, processed 2446581 words, keeping 

### Exploring the Model Results

In [44]:
model.doesnt_match("man woman child kitchen".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [46]:
model.doesnt_match("france england germany berlin".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'berlin'

In [47]:
model.doesnt_match("paris berlin london austria".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'austria'

In [48]:
model.most_similar("man")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6275639533996582),
 ('lady', 0.5752013921737671),
 ('lad', 0.5729827880859375),
 ('millionaire', 0.5349369049072266),
 ('men', 0.518455982208252),
 ('chap', 0.5147342681884766),
 ('monk', 0.5118533372879028),
 ('person', 0.5100691318511963),
 ('farmer', 0.5079906582832336),
 ('guy', 0.5078481435775757)]

In [49]:
model.most_similar("queen")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('princess', 0.6677519679069519),
 ('bride', 0.6282608509063721),
 ('maid', 0.5858337879180908),
 ('stepmother', 0.5829166173934937),
 ('carmen', 0.5773953795433044),
 ('goddess', 0.5764511823654175),
 ('latifah', 0.5732284188270569),
 ('mistress', 0.5730939507484436),
 ('aurora', 0.5722073316574097),
 ('maria', 0.5596499443054199)]

In [50]:
model.most_similar("awful")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.7837287187576294),
 ('horrible', 0.7368273735046387),
 ('atrocious', 0.7090774774551392),
 ('abysmal', 0.7000053524971008),
 ('dreadful', 0.691450834274292),
 ('appalling', 0.6747241020202637),
 ('horrendous', 0.6685676574707031),
 ('horrid', 0.663025438785553),
 ('lousy', 0.6215429306030273),
 ('crappy', 0.608646035194397)]